In [1]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

practice_16_trading_path = kagglehub.competition_download('practice-16-trading')

print('Data source import complete.')


100%|██████████| 581M/581M [00:22<00:00, 27.4MB/s]

Extracting files...


Data source import complete.


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Ваш номер ИСУ**

409682

In [5]:
import os
import pandas as pd

# Example: Load a day of training data
train_path = "/kaggle/input/practice-16-trading/train/"
sample_day = pd.read_csv(os.path.join(train_path, "2024-06-20.csv"), dtype="object")
operation_columns = ['date', 'buy_row', 'sell_row', 'instrument(call/put)', 'buy_price', 'sell_price']

# --- Define a placeholder trading strategy ---
class MyTradingStrategy:
    def __init__(self):
        pass

    def train(self, train_folder):
        # TODO: Train your model using historical data
        pass

    def predict(self, day_data, date: str) -> pd.DataFrame:
        # TODO: Make a decision: 'buy', 'sell', or 'hold'
        operations = []

        import random

        instruments = {col.split(".")[0] for col in day_data.columns if col.endswith("ask") or col.endswith("bid")}
        for _ in range(random.randint(1, 5)):
            buy_row = random.randint(0, 50)
            sell_row = random.randint(buy_row + 1, 60)
            instrument = random.choice(list(instruments))
            buy_price = round(random.uniform(10, 20), 2)
            sell_price = round(buy_price + random.uniform(0.1, 5), 2)
            operations.append([date, buy_row, sell_row, instrument, buy_price, sell_price])

        return pd.DataFrame(data=operations, columns = operation_columns)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/practice-16-trading/train/2024-06-20.csv'

# --- Evaluation function ---

### Функция, которая поможет скомпоновать `pd.DataFrame` в формате, нужном для отправки результата

In [ ]:
def combine_row_values_in_one_array_string(df : pd.DataFrame):
    def concat_rows(group):
        group_list = group.astype(str)[1:]
        return '|'.join(group_list)

    df["operations"] = df.apply(concat_rows, axis=1)
    df = df.groupby("date")["operations"].apply(lambda x: '|'.join(x)).reset_index()
    df["operations"] = df["operations"].apply(lambda x: x.split('|'))
    df["ID"] = df.index
    return  df

### Функция получения списка операций для всех дней из множества

In [ ]:
def apply_strategy_to_get_operations(strategy, testing_folder):
    path = f"/kaggle/input/practice-16-trading/{testing_folder}/"

    dates = os.listdir(path)
    dates = [date.split('.')[0] for date in dates if date.endswith('.csv')]

    all_operations = []

    for day in dates:
        day_data = pd.read_csv(f'{path}/{day}.csv', low_memory=False)
        day_operations = strategy.predict(day_data, day)
        all_operations.append(day_operations)

    return pd.concat(all_operations, ignore_index=True)

### Функция, которая поможет подсчитать Ваш получаемый результат до отправки. Но не стесняйтесь делать больше отправок (в день доступно 10).

In [ ]:
def evaluate_score(submission, testing_folder):
    total_profit = 0
    day_score = 0
    number_of_posititive_days = 0
    path = f"/kaggle/input/practice-16-trading/{testing_folder}/"
    total_number_of_days = len(os.listdir(path))

    dates = submission["date"].unique()

    for day in dates:
        try:
            day_data = pd.read_csv(f'{path}/{day}.csv', low_memory=False)
        except FileNotFoundError:
            raise ParticipantVisibleError(f"Day {day} should not be in your submission file.")

        operations = submission[submission['date'] == day]['operations'].values[0]

        for i in range(0, len(operations), 5):
            operation = operations[i:i+5]
            if len(operation) < 5:
                raise ParticipantVisibleError
            start_row, end_row, instrument, _, _ = operation

            buy_price = day_data.loc[int(start_row), f"{instrument}.0_ask"]
            sell_price = day_data.loc[int(end_row), f"{instrument}.0_bid"]

            profit = sell_price - buy_price
            day_score += profit

        # print(f"day={day} day_score={day_score}")
        if day_score > 0:
            number_of_posititive_days += 1
            day_score = 0.0

    print("Total Profit:", profit)
    print("Non-negative Days:", number_of_posititive_days)

    return number_of_posititive_days / total_number_of_days

# --- Run evaluation ---

In [ ]:
train_folder = "train"
test_folder = "test"  # hidden from users

strategy = MyTradingStrategy()
strategy.train(train_folder)
df = apply_strategy_to_get_operations(strategy, test_folder)

In [ ]:
df1 = combine_row_values_in_one_array_string(df)

In [ ]:
score = evaluate_score(df1, test_folder)
df.to_csv("submission.csv") # Записывает файл, который нужно отправить в качестве решения
print("Final Score:", score)